In [1]:
import ROOT
from ROOT import gStyle
import numpy as np
import ctypes
import os
import pandas as pd

def biner(edges,bin_widths,histogram):
    if (len(edges)+1!=len(bin_widths)):
        print("Check edges and bin widths array sizes!")
        return
    
    bins=[]
    first_bin = histogram.GetXaxis().GetBinLowEdge(1)
    last_bin = histogram.GetXaxis().GetBinUpEdge(histogram.GetNbinsX())
    for i in range(0,len(edges)):
        n_spaces = int((edges[i] - first_bin)/bin_widths[i])
        bins = np.concatenate((bins,np.linspace(first_bin,edges[i],n_spaces,endpoint=False)))
        first_bin = edges[i]
        if edges[i]==edges[-1]:
            n_spaces = int((last_bin - edges[i])/bin_widths[i+1])
            bins = np.concatenate((bins,np.linspace(edges[i],last_bin,n_spaces,endpoint=False)))
            bins = np.concatenate((bins,[last_bin]))
    return bins

def normalization(hist_list,norm_bin):
    for hist in hist_list:
        for i in range(1,hist.GetNbinsX()+1):
            value=hist.GetBinContent(i)
            error=hist.GetBinError(i)
            sf=hist.GetBinWidth(i)/norm_bin
            hist.SetBinContent(i,value/sf)
            hist.SetBinError(i,error/sf)

Welcome to JupyROOT 6.26/02


In [4]:
##### SELECTING GENERATOR FOR SIGNAL SAMPLE
signal_sample="Signal_PoPy.root"
production=True
#### CREATING THE CORRESPONDING MC.ROOT FILE EVERYTIME YOU CHANGE THE SIGNAL SAMPLE

os.system("rm MC.root")
os.system("hadd MC.root "+"BG.root "+signal_sample)

0

hadd Target file: MC.root
hadd compression setting for all output: 1
hadd Source file 1: BG.root
hadd Source file 2: Signal_PoPy.root
hadd Target path: MC.root:/


In [5]:
histos_no_rebin={"lep1_phi_topo":[[[1],[1,1],1],"#phi(#mu_{1})"],
"lep1_phi_cuts":[[[1],[1,1],1],"#phi(#mu_{1})"],
"lep2_phi_topo":[[[1],[1,1],1],"#phi(#mu_{2})"],
"lep2_phi_cuts":[[[1],[1,1],1],"#phi(#mu_{2})"],
"jet_n_topo":[[[1],[1,1],1],"jetN"],
"jet_n_topo_dphi_btag_iso_pt1_pt2_mass":[[[1],[1,1],1],"jetN"],
"jet_n_topo_dphi_btag_iso_pt1_pt2_mass_ptl":[[[1],[1,1],1],"jetN"],
"delta_phi":[[[1],[1,1],1],"#Delta#phi(#mu_{1},#mu_{2})"],
"delta_phi_topo":[[[1],[1,1],1],"#Delta#phi(#mu_{1},#mu_{2})"],
"delta_phi_cuts_butphi":[[[1],[1,1],1],"#Delta#phi(#mu_{1},#mu_{2})"],
"ratio_ptjet_zpt_cuts":[[[1],[1,1],1],"r_{pT(j)/pT(Z)}"],
"ratio_lpt_tpt_cuts":[[[1],[1,1],1],"r_{pT(#mu)/pT(#tau)}"],
"ratio_ptjet_zpt_cuts_ptl":[[[1],[1,1],1],"r_{pT(j)/pT(Z)}"],
"ratio_lpt_tpt_cuts_ptl":[[[1],[1,1],1],"r_{pT(#mu)/pT(#tau)}"],

"sum_pt_pt_cuts_ZpTa":[[[1],[1,1],1],"p_{T}(Z)(GeV)(i)"],
"sum_pt_pt_cuts_ZpTb":[[[1],[1,1],1],"p_{T}(Z)(GeV)(i)"],
"sum_pt_pt_cuts_ZpTc":[[[1],[1,1],1],"p_{T}(Z)(GeV)(i)"],
"sum_pt_pt_cuts_ptl_ZpTa":[[[1],[1,1],1],"p_{T}(Z)(GeV)(i)"],
"sum_pt_pt_cuts_ptl_ZpTb":[[[1],[1,1],1],"p_{T}(Z)(GeV)(i)"],
"sum_pt_pt_cuts_ptl_ZpTc":[[[1],[1,1],1],"p_{T}(Z)(GeV)(i)"],}

histos_rebin={"lep1_pt_topo":[[[180],[2,2],2],"p_{T}(#mu_{1})(GeV)"],
"lep1_pt_topo_dphi_btag_iso_pt2_mass_ptl":[[[180],[2,2],2],"p_{T}(#mu_{1})(GeV)"],
"lep2_pt_topo":[[[180],[2,2],2],"p_{T}(#mu_{2})(GeV)"],
"lep2_pt_topo_dphi_btag_iso_pt1_mass_ptl":[[[180],[2,2],2],"p_{T}(#mu_{2})(GeV)"],
"sumlep_pt_topo":[[[180],[1,10],1],"#sum p_{T}(GeV)"],
"sumlep_pt_topo_dphi_btag_iso_pt1_pt2_mass_ptl":[[[180],[5,5],5],"#sum p_{T}(GeV)"],
"MET_topo":[[[120],[5,5],5],"MET(GeV)"],
"MET_topo_dphi_btag_iso_pt1_pt2_mass_ptl":[[[120],[5,5],5],"MET(GeV)"],
"inv_mass_topo":[[[120],[5,5],5],"m(#mu_{1},#mu_{2})(GeV)"],
"h_inv_mass_topo_dphi_btag_iso_pt1_pt2_ptl":[[[120],[5,10],5],"m(#mu_{1},#mu_{2})(GeV)"],
"Z_pt_topo":[[[260],[10,20],10],"Zp_{T}(GeV)"],
"Z_pt_cuts":[[[260],[10,20],10],"Zp_{T}(GeV)"],
"Z_pt_cuts_ptl":[[[260],[10,20],10],"Zp_{T}(GeV)"],
"ljet1_pt_topo_cuts":[[[120],[5,20],5],"p_{T}(jet_{1})(GeV)"],
"ljet1_pt_topo_cuts_tpt":[[[120],[5,20],5],"p_{T}(jet_{1})(GeV)"],
"ljet2_pt_topo_cuts":[[[120],[5,20],5],"p_{T}(jet_{2})(GeV)"],
"ljet2_pt_topo_cuts_tpt":[[[120],[5,20],5],"p_{T}(jet_{2})(GeV)"],


"lep1_eta_cuts":[[[-1.3,-0.1,0.1,1.3],[0.399999,0.399999,0.2,0.399999,0.399999],0.4],"#eta(#mu_{1})",1.2,2.5],
"lep2_eta_cuts":[[[-1.3,-0.1,0.1,1.3],[0.399999,0.399999,0.2,0.399999,0.399999],0.4],"#eta(#mu_{2})",1.2,2.5],  

"lep1_eta_cuts_ptl":[[[-1.3,-0.1,0.1,1.3],[0.399999,0.399999,0.2,0.399999,0.399999],0.4],"#eta(#mu_{1})",1.2,2.5],
"lep2_eta_cuts_ptl":[[[-1.3,-0.1,0.1,1.3],[0.399999,0.399999,0.2,0.399999,0.399999],0.4],"#eta(#mu_{2})",1.2,2.5],   

"tau_eta_topo_cuts":[[[-1.3,-0.1,0.1,1.3],[0.399999,0.399999,0.2,0.399999,0.399999],0.4],"#eta(#mu)"],
"tau_pt_topo_cuts":[[[120],[5,20],5],"p_{T}(#mu)"],
"lep1_eta_loose":[[[-1.3,-0.1,0.1,1.3],[0.399999,0.399999,0.2,0.399999,0.399999],0.4],"#eta(#mu)"],              
"lep2_eta_loose":[[[-1.3,-0.1,0.1,1.3],[0.399999,0.399999,0.2,0.399999,0.399999],0.4],"#eta(#mu)"],              
}

histos={}
histos.update(histos_no_rebin)
histos.update(histos_rebin)
    
    
for i in histos:
    print(i)
    MC=ROOT.TFile.Open("MC.root","READ")
    Data = ROOT.TFile.Open("Data.root","READ")
    Signal=ROOT.TFile.Open(signal_sample,"READ")
    VV=ROOT.TFile.Open("VV.root","READ")
    Wjets=ROOT.TFile.Open("Wjets.root","READ")
    Zjets=ROOT.TFile.Open("Ztautau_PoPy.root","READ")
    Ttbar=ROOT.TFile.Open("ttbar.root","READ")
    SingleTop=ROOT.TFile.Open("singletop.root","READ")
    Error=ROOT.TFile.Open("MC.root","READ")
    
    gStyle.SetOptStat(1111111)
    a=i
    
    mc=MC.Get(a)
    data=Data.Get(a)
    signal=Signal.Get(a)
    vv=VV.Get(a)
    w=Wjets.Get(a)
    z=Zjets.Get(a)
    tt=Ttbar.Get(a)
    t=SingleTop.Get(a)
    error=Error.Get(a)
    
    mc.SetDirectory(0)
    signal.SetDirectory(0)
    data.SetDirectory(0)
    vv.SetDirectory(0)
    w.SetDirectory(0)
    z.SetDirectory(0)
    tt.SetDirectory(0)
    t.SetDirectory(0)
    error.SetDirectory(0)
    
    MC.Close()
    Signal.Close()
    Data.Close()
    VV.Close()
    Wjets.Close()
    Zjets.Close()
    Ttbar.Close()
    SingleTop.Close()
    Error.Close()

    hs = ROOT.THStack("hs","")
    
    s=data.GetXaxis().GetBinLowEdge(1)
    e=data.GetXaxis().GetBinUpEdge(data.GetNbinsX())
    
    if i in histos_rebin:
        ob=data.GetNbinsX()
        s=data.GetXaxis().GetBinLowEdge(1)
        e=data.GetXaxis().GetBinUpEdge(data.GetNbinsX())
        rebining=biner(histos[i][0][0],histos[i][0][1],data)
        nb=len(rebining)-1
        mc=mc.Rebin(nb,"mc",rebining)
        signal=signal.Rebin(nb,"signal",rebining)
        data=data.Rebin(nb,"data",rebining)
        vv=vv.Rebin(nb,"vv",rebining)
        tt=tt.Rebin(nb,"tt",rebining)
        t=t.Rebin(nb,"t",rebining)
        z=z.Rebin(nb,"z",rebining)
        w=w.Rebin(nb,"w",rebining)
        error=error.Rebin(nb,"error",rebining)
    
        hist_list=[signal,data,vv,tt,t,z,w,mc,error]
        normalization(hist_list,histos[i][0][2])



    hs.Add(vv)
    hs.Add(w)
    hs.Add(z)
    hs.Add(tt)
    hs.Add(t)
    hs.Add(signal)

    ratio=mc.Clone()
    ratio.Divide(data)
    ratio_error=ratio.Clone()
    
    signal.SetFillColor(ROOT.kAzure+8)
    w.SetFillColor(ROOT.kGreen)
    w.SetLineColor(ROOT.kGreen+1)
    z.SetFillColor(ROOT.kRed)
    z.SetLineColor(ROOT.kRed+1)
    vv.SetFillColor(ROOT.kBlue)
    vv.SetLineColor(ROOT.kBlue+1)
    tt.SetFillColor(ROOT.kYellow)
    tt.SetLineColor(ROOT.kYellow+1)
    t.SetFillColor(ROOT.kCyan)
    t.SetLineColor(ROOT.kCyan+1)
    data.SetLineColor(ROOT.kBlack)
    error.SetFillColorAlpha(ROOT.kBlue,0.5)
    ratio_error.SetFillColorAlpha(ROOT.kBlue,0.5)

    gStyle.SetOptStat(1111111)

    canvas3 = ROOT.TCanvas("canvas3")
    canvas3.cd()

    ##### DRAWING TOP PAD, SETTING MARGINS #######

    pad1 = ROOT . TPad (" pad1 "," pad1 " ,0 ,0.35 ,1 ,1)
    pad1.SetTopMargin(0.03)
    pad1.SetRightMargin(0.03)
    pad1.SetLeftMargin(0.08)
    pad1.SetBottomMargin(0.0)
    pad1.Draw ()
    pad1.cd ()

    ###### SETTING STATS BOX POSITION ######
    
    gStyle.SetStatY(0.93);                
    gStyle.SetStatX(0.95);
    gStyle.SetStatW(0.1);                
    gStyle.SetStatH(0.1);


    data.Draw("pe")
    hs.Draw("HIST same")
    error.Draw("E2 same")
    if "inv_mass" in i:
        pad1.SetLogy()
        s+=50
        
    if(not production):
    
    ##### DRAWING THE STATISTICS BOX #####
    
        gStyle.SetOptStat(1111111)
    
    ###### SETTING STATS BOX POSITION ######
    
        gStyle.SetStatY(0.97);                
        gStyle.SetStatX(0.20);
        gStyle.SetStatW(0.12);                
        gStyle.SetStatH(0.12);
        
    ###### ADDING A LABEL ACCORDING TO THE CHANNEL ######
    
        data.SetTitle("")
        l=ROOT.TLatex()
        l.SetNDC ()
        l.SetTextSize(0.125)
        l.DrawLatex(0.80,0.85,"Z#rightarrow ee")
        
    else :
        gStyle.SetOptStat(0)
        data.SetTitle("")
        l=ROOT.TLatex()
        l.SetNDC ()
        l.SetTextSize(0.062)
        l.DrawLatex(0.12,0.85,"Z#rightarrow ee")


    data . GetYaxis (). SetRangeUser (1 ,1.2*data.GetBinContent(data.GetMaximumBin()))
    data . GetXaxis (). SetRangeUser (s,e)

    hs . GetXaxis (). SetRangeUser (20e3 ,30e4)
    hs . GetYaxis (). SetRangeUser (0 ,40)
    legend = ROOT . TLegend (0.53 ,0.80 ,0.96 ,0.95)
    legend . AddEntry ( vv ,"Diboson")
    legend . AddEntry ( data ," Data ")
    legend . AddEntry ( signal ," Signal ")
    legend . AddEntry ( w ," W+jets ")
    legend . AddEntry ( z ," Z+jets ")
    legend . AddEntry ( t ," Single top ")
    legend . AddEntry ( tt ," ttbar ")
    legend.SetTextAlign(22)
    legend.SetNColumns(3)
    gStyle.SetLegendBorderSize(0)
    legend . SetLineWidth (2)
    legend . Draw ()

    canvas3.cd()
    pad2 = ROOT . TPad (" pad2 "," pad2 " ,0 ,0.00 ,1 ,0.35)
    pad2.SetRightMargin(0.03)
    pad2.SetLeftMargin(0.08)
    pad2.SetTopMargin(0)
    pad2.SetBottomMargin(0.2)
    pad2.Draw ()
    pad2.cd ()
    ratio.SetMarkerStyle(ROOT.kFullCircle)
    ratio.SetMarkerSize(0.7)
    ratio.Draw ("p HIST")
    ratio_error.Draw ("E2 same")
    ratio.Draw ("p hist same")
    ratio.SetTitle("")
    ratio.SetStats(0)
    ratio . GetYaxis (). SetRangeUser (0.5 ,1.5)
    ratio . GetXaxis (). SetRangeUser (s ,e)
    ratio . GetYaxis (). SetTitle ("MC/DATA")
    ratio . GetYaxis (). SetTitleSize (0.15)
    ratio . GetYaxis (). SetTitleOffset (0.25)
    ratio . GetXaxis (). SetTitle (histos[i][1])
    ratio . GetXaxis (). SetTitleSize (0.09)
    ratio.GetXaxis().SetLabelSize(0.10)
    ratio.GetYaxis().SetLabelSize(0.08)
        

    ###### SETTING ALL THE HORIZONTAL DASHED LINES #######
    
    line = ROOT . TLine (s ,1 ,e,1)
    line . SetLineColor ( ROOT . kBlack )
    line . SetLineWidth (2)
    line2 = ROOT . TLine (s ,1.1 ,e,1.1)
    line2 . SetLineColor ( ROOT . kBlack )
    line2 . SetLineWidth (1)
    line2 . SetLineStyle(2)
    line3 = ROOT . TLine (s ,0.9 ,e,0.9)
    line3 . SetLineColor ( ROOT . kBlack )
    line3 . SetLineWidth (1)
    line3 . SetLineStyle(2)
    line4 = ROOT . TLine (s ,1.2 ,e,1.2)
    line4 . SetLineColor ( ROOT . kBlack )
    line4 . SetLineWidth (1)
    line4 . SetLineStyle(2)
    line5 = ROOT . TLine (s ,0.8 ,e,0.8)
    line5 . SetLineColor ( ROOT . kBlack )
    line5 . SetLineWidth (1)
    line5 . SetLineStyle(2)
    line6 = ROOT . TLine (s ,1.3 ,e,1.3)
    line6 . SetLineColor ( ROOT . kBlack )
    line6 . SetLineWidth (1)
    line6 . SetLineStyle(2)
    line7 = ROOT . TLine (s ,0.7 ,e,0.7)
    line7 . SetLineColor ( ROOT . kBlack )
    line7 . SetLineWidth (1)
    line7 . SetLineStyle(2)
    line8 = ROOT . TLine (s ,1.4 ,e,1.4)
    line8 . SetLineColor ( ROOT . kBlack )
    line8 . SetLineWidth (1)
    line8 . SetLineStyle(2)
    line9 = ROOT . TLine (s ,0.6 ,e,0.6)
    line9 . SetLineColor ( ROOT . kBlack )
    line9 . SetLineWidth (1)
    line9 . SetLineStyle(2)
    line . Draw (" same ")
    line2 . Draw (" same ")
    line3 . Draw (" same ")
    line4 . Draw (" same ")
    line5 . Draw (" same ")
    line6 . Draw (" same ")
    line7 . Draw (" same ")
    line8 . Draw (" same ")
    line9 . Draw (" same ")
    line . Draw (" same ")





    canvas3.Print(a+signal_sample.strip('Signal')+".pdf")
    if i=="ratio_lpt_tpt_cuts_ptl":
        dic={"data":data,"signal":signal,"vv":vv,"w":w,"z":z,"tt":tt,"t":t}
        dic_numbers={"data":[0,0],"signal":[0,0],"vv":[0,0],"w":[0,0],"z":[0,0],"tt":[0,0],"t":[0,0]}
        for i in dic:
            error=ctypes.c_double()
            a=dic[i].IntegralAndError(0,1000,error)
            dic_numbers[i][0]=a
            dic_numbers[i][1]=error.value
        dataframe=pd.DataFrame.from_dict(dic_numbers, orient='index',columns=['Yield', 'Stat. Uncer'])
        display(dataframe.round(decimals=3))




lep1_phi_topo
lep1_phi_cuts
lep2_phi_topo
lep2_phi_cuts
jet_n_topo
jet_n_topo_dphi_btag_iso_pt1_pt2_mass
jet_n_topo_dphi_btag_iso_pt1_pt2_mass_ptl
delta_phi
delta_phi_topo
delta_phi_cuts_butphi
ratio_ptjet_zpt_cuts
ratio_lpt_tpt_cuts
ratio_ptjet_zpt_cuts_ptl
ratio_lpt_tpt_cuts_ptl


,Yield,Stat. Uncer
data,170783.000,413.259
signal,162477.447,219.923
vv,3813.414,12.897
w,8.399,8.399
z,50.815,10.931
tt,134.426,4.393
t,18.488,1.584


sum_pt_pt_cuts_ZpTa
sum_pt_pt_cuts_ZpTb
sum_pt_pt_cuts_ZpTc
sum_pt_pt_cuts_ptl_ZpTa
sum_pt_pt_cuts_ptl_ZpTb
sum_pt_pt_cuts_ptl_ZpTc
lep1_pt_topo
lep1_pt_topo_dphi_btag_iso_pt2_mass_ptl
lep2_pt_topo
lep2_pt_topo_dphi_btag_iso_pt1_mass_ptl
sumlep_pt_topo
sumlep_pt_topo_dphi_btag_iso_pt1_pt2_mass_ptl
MET_topo
MET_topo_dphi_btag_iso_pt1_pt2_mass_ptl
inv_mass_topo
h_inv_mass_topo_dphi_btag_iso_pt1_pt2_ptl
Z_pt_topo
Z_pt_cuts
Z_pt_cuts_ptl
ljet1_pt_topo_cuts
ljet1_pt_topo_cuts_tpt
ljet2_pt_topo_cuts
ljet2_pt_topo_cuts_tpt
lep1_eta_cuts
lep2_eta_cuts
lep1_eta_cuts_ptl
lep2_eta_cuts_ptl
tau_eta_topo_cuts
tau_pt_topo_cuts
lep1_eta_loose
lep2_eta_loose


Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file lep1_phi_topo_PoPy.root.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file lep1_phi_cuts_PoPy.root.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file lep2_phi_topo_PoPy.root.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file lep2_phi_cuts_PoPy.root.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file jet_n_topo_PoPy.root.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file jet_n_topo_dphi_btag_iso_pt1_pt2_mass_PoPy.root.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canva

### 

In [9]:
def print_yields(first_bin,last_bin,histogram_name):
            dic={"data":"Data.root","signal":signal_sample,"vv":"VV.root","w":"Wjets.root","z":"Ztautau_PoPy.root","tt":"ttbar.root","t":"singletop.root"}
            dic_numbers={"data":[0,0],"signal":[0,0],"vv":[0,0],"w":[0,0],"z":[0,0],"tt":[0,0],"t":[0,0]}
            for i in dic:
                filename = str(dic[i])
                file = ROOT.TFile.Open(filename)
                
                histo_1d = file.Get(histogram_name)
                error=ctypes.c_double()
                Yield=histo_1d.IntegralAndError(first_bin,last_bin,error)
                Error = error.value
                dic_numbers[i][0]=Yield
                dic_numbers[i][1]=Error
                
                file.Close()

            dataframe=pd.DataFrame.from_dict(dic_numbers, orient='index',columns=['Yield', 'Stat. Uncer'])
            display(dataframe.round(decimals=3))


In [10]:
print_yields(1,77,"tau_pt_topo_cuts") #78->46

,Yield,Stat. Uncer
data,131284.000,362.331
signal,126678.323,190.518
vv,2181.074,10.304
w,0.000,0.000
z,34.810,9.084
tt,268.795,6.207
t,36.327,2.204


In [11]:
eta = {"lep1_eta_cuts_ptl":[1,12,13,25,26,38,39,50],"tau_pt_topo_cuts":[1,45,46,65,66,201,46,201]}
for i in eta:
    print(i)
    for c in range(len(eta[i])):
        if c%2==0:
            print_yields(eta[i][c],eta[i][c+1],str(i))

lep1_eta_cuts_ptl


,Yield,Stat. Uncer
data,21957.000,148.179
signal,20620.715,76.989
vv,425.641,4.239
w,0.000,0.000
z,3.790,2.938
tt,11.903,1.310
t,0.948,0.362


,Yield,Stat. Uncer
data,63140.000,251.277
signal,60253.896,134.428
vv,1477.195,7.875
w,8.399,8.399
z,20.912,7.033
tt,59.895,2.950
t,7.872,1.043


,Yield,Stat. Uncer
data,63662.000,252.313
signal,60575.593,134.995
vv,1485.390,8.264
w,0.000,0.000
z,23.855,7.503
tt,51.571,2.708
t,8.501,1.066


,Yield,Stat. Uncer
data,22024.000,148.405
signal,21027.247,78.386
vv,425.142,4.252
w,0.000,0.000
z,2.258,2.258
tt,11.057,1.245
t,1.167,0.393


tau_pt_topo_cuts


,Yield,Stat. Uncer
data,0.0,0.0
signal,0.0,0.0
vv,0.0,0.0
w,0.0,0.0
z,0.0,0.0
tt,0.0,0.0
t,0.0,0.0


,Yield,Stat. Uncer
data,84764.000,291.143
signal,81961.638,152.880
vv,1377.793,8.396
w,0.000,0.000
z,20.763,7.005
tt,190.752,5.228
t,27.475,1.925


,Yield,Stat. Uncer
data,221696.000,470.846
signal,211258.114,249.870
vv,4684.238,14.318
w,8.399,8.399
z,67.435,12.631
tt,221.984,5.653
t,28.499,1.955


,Yield,Stat. Uncer
data,306460.000,553.588
signal,293219.752,292.929
vv,6062.031,16.598
w,8.399,8.399
z,88.198,14.444
tt,412.736,7.699
t,55.974,2.744
